In [1]:
from pinecone import Pinecone, ServerlessSpec
import os
from openai import OpenAI
import pandas as pd
from time import time
import dotenv
dotenv.load_dotenv()

True

In [2]:
token= os.getenv("RUNPOD_TOKEN") 
open_ai_base_url = os.getenv("RUNPOD_EMBEDDING_URL") 
model_name= os.getenv("MODEL_NAME") 
pinecone_api_key = os.getenv("PINECONE_API_KEY") 

In [4]:
pc = Pinecone(api_key=pinecone_api_key)

client = OpenAI(
  api_key=token, 
  base_url=open_ai_base_url
)

## Try out embeddings

In [5]:
output = client.embeddings.create(input = ["helloo there"],model=model_name)
embedings = output.data[0].embedding
print(embedings)

[-0.055366937071084976, -0.056556638330221176, 0.08584163337945938, -0.06238313391804695, 0.01755574159324169, -0.01140131801366806, 0.052316416054964066, 0.05259096249938011, 0.028903676196932793, -0.022573847323656082, -0.013201124966144562, -0.04783215373754501, 0.029452770948410034, 0.031847428530454636, 0.056312598288059235, -0.008991407230496407, 0.012819809839129448, -0.05423824489116669, -0.10365666449069977, -0.019553832709789276, 0.02913246490061283, 0.05204186961054802, -0.02882741391658783, -0.034775927662849426, 0.001531932968646288, -0.00663869408890605, 0.019736863672733307, 0.03556906431913376, -0.008488072082400322, -0.07382258027791977, -0.0005252613918855786, -0.012873194180428982, 0.046581439673900604, -0.00035176306846551597, 0.05137075483798981, 0.0004532882012426853, 0.05960715934634209, -0.025029515847563744, -0.07791028171777725, -0.005586264654994011, 0.060491811484098434, -0.02512103132903576, -0.0016205887077376246, -0.016579575836658478, 0.02251283638179302

In [6]:
len(embedings)

384

## Wrangle dataset

In [7]:
df=pd.read_json('products/products.jsonl',lines=True)

In [8]:
df.head(2)

,name,category,description,ingredients,price,rating,image_path
0,Cappuccino,Coffee,A rich and creamy cappuccino made with freshly...,"[Espresso, Steamed Milk, Milk Foam]",4.50,4.7,cappuccino.jpg
1,Jumbo Savory Scone,Bakery,"Deliciously flaky and buttery, this jumbo savo...","[Flour, Butter, Cheese, Herbs, Baking Powder, ...",3.25,4.3,SavoryScone.webp


In [9]:
df['text'] =  df['name']+" : "+df['description'] + \
                " -- Ingredients: " + df['ingredients'].astype(str) + \
                " -- Price: " + df['price'].astype(str) + \
                " -- rating: " + df['rating'].astype(str) 

In [10]:
df['text'].head()

0    Cappuccino : A rich and creamy cappuccino made...
1    Jumbo Savory Scone : Deliciously flaky and but...
2    Latte : Smooth and creamy, our latte combines ...
3    Chocolate Chip Biscotti : Crunchy and delightf...
4    Espresso shot : A bold shot of rich espresso, ...
Name: text, dtype: object

In [11]:
texts = df['text'].tolist()

In [12]:
texts[:2]

["Cappuccino : A rich and creamy cappuccino made with freshly brewed espresso, steamed milk, and a frothy milk cap. This delightful drink offers a perfect balance of bold coffee flavor and smooth milk, making it an ideal companion for relaxing mornings or lively conversations. -- Ingredients: ['Espresso', 'Steamed Milk', 'Milk Foam'] -- Price: 4.5 -- rating: 4.7",
 "Jumbo Savory Scone : Deliciously flaky and buttery, this jumbo savory scone is filled with herbs and cheese, creating a mouthwatering experience. Perfect for a hearty snack or a light lunch, it pairs beautifully with your favorite coffee or tea. -- Ingredients: ['Flour', 'Butter', 'Cheese', 'Herbs', 'Baking Powder', 'Salt'] -- Price: 3.25 -- rating: 4.3"]

In [14]:
with open('products/Merry\'s_way_about_us.txt') as f:
    Merry_way_about_section = f.read()
    
Merry_way_about_section = "Coffee shop Merry's Way about section: " + Merry_way_about_section
texts.append(Merry_way_about_section)

In [15]:
Merry_way_about_section

"Coffee shop Merry's Way about section: Welcome to Merry's Way Coffee, your neighborhood coffee shop located in the heart of Greenwich Village, New York City. At Merry's Way, we believe that coffee is more than just a drink—it’s an experience, a moment of joy, and a way to connect with others.\n\nOur Story\nFounded in 2015, Merry’s Way started as a small family-owned café with one mission: to share the love of quality, ethically-sourced coffee with our community.\n\nMerry's passion for travel and coffee led her on a journey across South America, where she handpicked partnerships with small farms and cooperatives. We ensure that every cup we brew tells a story of dedication and care, from farm to table. Our beans are roasted in-house to bring out unique flavors that reflect the regions where they were grown.\n\nDelivery & Locations Served\nIn addition to offering a cozy place to enjoy coffee in our café, we proudly deliver to Greenwich Village, SoHo, West Village, and Lower Manhattan. W

In [16]:
with open('products/menu_items_text.txt') as f:
    menue_items_text = f.read()
    
menue_items_text = "Menu Items: " + menue_items_text
texts.append(menue_items_text)

In [17]:
menue_items_text

'Menu Items: Menu Items\n\nCappuccino - $4.50\nJumbo Savory Scone - $3.25\nLatte - $4.75\nChocolate Chip Biscotti - $2.50\nEspresso shot - $2.00\nHazelnut Biscotti - $2.75\nChocolate Croissant - $3.75\nDark chocolate (Drinking Chocolate) - $5.00\nCranberry Scone - $3.50\nCroissant - $3.25\nAlmond Croissant - $4.00\nGinger Biscotti - $2.50\nOatmeal Scone - $3.25\nGinger Scone - $3.50\nChocolate syrup - $1.50\nHazelnut syrup - $1.50\nCarmel syrup - $1.50\nSugar Free Vanilla syrup - $1.50\nDark chocolate (Packaged Chocolate) - $3.00'

## Generate Embeddings

In [18]:
output = client.embeddings.create(input = texts,model=model_name)

In [19]:
embeddings = output.data

In [20]:
embeddings[0]

Embedding(embedding=[-0.014078277163207531, -0.06446248292922974, 0.003805713029578328, 0.004784325137734413, 0.030125223100185394, -0.045630402863025665, 0.03723685070872307, 0.010690334253013134, -0.027744507417082787, -0.05091071128845215, -0.052436813712120056, -0.03366577625274658, -0.0004516303597483784, -0.043768562376499176, 0.03757259249687195, -0.01718389242887497, -0.019488302990794182, -0.08063914626836777, -0.07587771862745285, -0.007950983941555023, 0.0483773872256279, -0.06122715026140213, -0.10749851912260056, -0.03699267655611038, 0.03549709916114807, -0.04856051877140999, 0.08790338784456253, -0.01983930729329586, 0.001731170224957168, -0.13075628876686096, -0.012933701276779175, 0.009805195964872837, -0.02801920473575592, -0.025821620598435402, -0.0350392684340477, 0.0010091339936479926, 0.06751468777656555, -0.07593876123428345, 0.047095462679862976, 0.0005737183964811265, -0.002592463279142976, 0.029804741963744164, -0.06415726244449615, -0.002075496595352888, -0.0

In [21]:
len(embedings)

384

## Push data to database

In [22]:
index_name = "coffeeshop"

pc.create_index(
    name=index_name,
    dimension=384, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

In [23]:
# Wait for the index to be ready
while not pc.describe_index(index_name).status['ready']:
    time.sleep(1)

index = pc.Index(index_name)

vectors = []
for text, e in zip(texts, embeddings):
    entry_id = text.split(":")[0].strip()
    vectors.append({
        "id": entry_id,
        "values": e.embedding,
        "metadata": {'text': text}
    })
    
index.upsert(
    vectors=vectors,
    namespace="ns1"
)

{'upserted_count': 20}

## Get Closest documents

In [24]:
output = client.embeddings.create(input = ["Is Cappuccino lactose-free?"],model=model_name)
embeding = output.data[0].embedding

In [25]:
results = index.query(
    namespace="ns1",
    vector=embeding,
    top_k=3,
    include_values=False,
    include_metadata=True
)

print(results)

{'matches': [{'id': 'Cappuccino',
              'metadata': {'text': 'Cappuccino : A rich and creamy cappuccino '
                                   'made with freshly brewed espresso, steamed '
                                   'milk, and a frothy milk cap. This '
                                   'delightful drink offers a perfect balance '
                                   'of bold coffee flavor and smooth milk, '
                                   'making it an ideal companion for relaxing '
                                   'mornings or lively conversations. -- '
                                   "Ingredients: ['Espresso', 'Steamed Milk', "
                                   "'Milk Foam'] -- Price: 4.5 -- rating: 4.7"},
              'score': 0.734829128,
              'values': []},
             {'id': 'Sugar Free Vanilla syrup',
              'metadata': {'text': 'Sugar Free Vanilla syrup : Enjoy the sweet '
                                   'flavor of vanilla without the 

In [26]:
source_knowledge = "\n".join([x['metadata']['text'].strip()+'\n' for x in results['matches'] ])

In [27]:
source_knowledge

"Cappuccino : A rich and creamy cappuccino made with freshly brewed espresso, steamed milk, and a frothy milk cap. This delightful drink offers a perfect balance of bold coffee flavor and smooth milk, making it an ideal companion for relaxing mornings or lively conversations. -- Ingredients: ['Espresso', 'Steamed Milk', 'Milk Foam'] -- Price: 4.5 -- rating: 4.7\n\nSugar Free Vanilla syrup : Enjoy the sweet flavor of vanilla without the sugar, making it perfect for your coffee or dessert. This syrup offers a guilt-free way to enhance your beverages, ensuring you never miss out on flavor. -- Ingredients: ['Water', 'Natural Flavors', 'Sucralose'] -- Price: 1.5 -- rating: 4.4\n\nMenu Items: Menu Items\n\nCappuccino - $4.50\nJumbo Savory Scone - $3.25\nLatte - $4.75\nChocolate Chip Biscotti - $2.50\nEspresso shot - $2.00\nHazelnut Biscotti - $2.75\nChocolate Croissant - $3.75\nDark chocolate (Drinking Chocolate) - $5.00\nCranberry Scone - $3.50\nCroissant - $3.25\nAlmond Croissant - $4.00\n

In [38]:
dict = {
    'text': 'Cappuccino : A rich and creamy cappuccino '
            'made with freshly brewed espresso, steamed '
            'milk, and a frothy milk cap. This '
            'delightful drink offers a perfect balance '
            'of bold coffee flavor and smooth milk, '
            'making it an ideal companion for relaxing '
            'mornings or lively conversations. -- '
            "Ingredients: ['Espresso', 'Steamed Milk', "
            "'Milk Foam'] -- Price: 4.5 -- rating: 4.7"
}

print(dict['text'])


Cappuccino : A rich and creamy cappuccino made with freshly brewed espresso, steamed milk, and a frothy milk cap. This delightful drink offers a perfect balance of bold coffee flavor and smooth milk, making it an ideal companion for relaxing mornings or lively conversations. -- Ingredients: ['Espresso', 'Steamed Milk', 'Milk Foam'] -- Price: 4.5 -- rating: 4.7


dict['text']
